In [8]:
from langdetect import detect, detect_langs
from nltk.corpus import XMLCorpusReader
import numpy as np
from nltk.corpus import XMLCorpusReader

def translate(x):
    try:
        t=detect(x)
    except:
        t=np.nan
    return t

We received feedback that the low number of german tweets might be because langdetect cannot properly detect the swiss german dialenct. To test this, we downloaded a swiss german corpus from http://kitt.cl.uzh.ch/kitt/noah/corpus. The first corpus is downloaded from swiss german blogs.

In [14]:
blick = XMLCorpusReader("." ,'blogs.xml')
swiss_german_text=" ".join(blick.words()).split(" . ")
swiss_german_text[0:5]

["s ' gaht nüme lang",
 'hallo mitenand',
 'zerscht mal : ich han en neue Blog müesse erstelle , will dr Andr gspunne het',
 'alsoo',
 'ich gah scho bald mal']

We use langdetect on each sentance and achieve a reasoably high accuracy.

In [16]:
float(sum(list(map(lambda x: translate(x)=='de',swiss_german_text))))/len(swiss_german_text)

0.8250510551395507

The accuracy is even higher on a swiss german newspaper from Zurich. This proves that the swiss german dialect is not an issue, but also warns us that langdetect might be less reliable on online (gramatically less correct) text such as twitter. 

In [17]:
blick = XMLCorpusReader("." ,'blick.xml')
swiss_german_text=" ".join(blick.words()).split(" . ")
float(sum(list(map(lambda x: translate(x)=='de',swiss_german_text))))/len(swiss_german_text)

0.9593984962406015